In [1]:
import torch
from pprint import pprint
from tqdm import tqdm
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
from utils.nlp import parse_state

In [34]:
device = "cuda"
tokenizer = GPT2Tokenizer.from_pretrained("models/gpt2/ta_encode/multiwoz",
                                            padding_side="left", truncation_side="left")
model = GPT2LMHeadModel.from_pretrained("models/gpt2/ta_encode/multiwoz")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50304, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [2]:
datasets = load_dataset("json", data_files={
        "train": "data/multiwoz/train/encoded.json",
        "valid": "data/multiwoz/dev/encoded.json",
        "test": "data/multiwoz/test/encoded.json",
    })

Using custom data configuration default-6eee9661713fc240


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /home/jader/.cache/huggingface/datasets/json/default-6eee9661713fc240/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def gen_examples()

In [35]:
sizencode = 256
predicted = {}
for batch in tqdm(datasets["test"]):
    did = batch["id"].lower().rstrip(".json")
    utterances = batch["text"].split("<sos_r>")
    predicted[did] = []
    responses = []
    for i in range(len(utterances)-1):
        example = "<sos_r>".join(utterances[:i+1])[-sizencode:]
        responses.append(example)
    encode = tokenizer(responses, return_tensors="pt", truncation=True,
                        padding=True, max_length=sizencode)
    encode = {k:v.to(device) for k,v in encode.items()}
    generate = model.generate(
        **encode,
        max_new_tokens=80,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.encode("<eos_r>")[0]
    )
    for gen in generate:
        gen = tokenizer.decode(gen)
        response = gen.split("<sos_r>")[-1].split("<eos_r>")[0].strip()
        state = {}
        for k,v in parse_state(gen.split("<sos_b>")[-1].split("<eos_b>")[0].strip()):
            state[k.split("_")[1]] = v
        predicted[did].append({
            "response": response,
            "state": state,
        })

100%|██████████| 1000/1000 [06:20<00:00,  2.63it/s]


In [51]:
predicted = {'pmul3520': [{'response': '[value_arrive] will work for you.',
   'state': {'train': {'departure': 'broxbourne',
     'destination': 'cambridge',
     'leave': '10:15'}},},]}

In [52]:
from mwzeval.metrics import Evaluator
e = Evaluator(bleu=True, success=True, richness=True)
results = e.evaluate(predicted)
print(results)

{'bleu': {'mwz22': 9.846052248031867}, 'success': {'inform': {'total': 0.0, 'train': 100.0, 'restaurant': 0.0}, 'success': {'total': 0.0, 'train': 0.0, 'restaurant': 0.0}}, 'richness': {'entropy': 2.321928094887362, 'cond_entropy': -0.0, 'avg_lengths': 5.0, 'msttr': 1.0, 'num_unigrams': 5, 'num_bigrams': 4, 'num_trigrams': 3}, 'dst': None}


In [53]:
predicted

{'pmul3520': [{'response': 'TIME will work for you.',
   'state': {'train': {'departure': 'broxbourne',
     'destination': 'cambridge',
     'leave': '10:15'}},
   'active_domains': ['train']}]}

In [5]:
from utils import dbPointer
dbPointer.queryResultVenues("train", {
                        "departure" :"broxbourne",
                        "destination": "cambridge",
                        "leaveat": "10:15"
                        }, True)[0]

select * from train where  departure='broxbourne' and destination='cambridge' and leaveat > '10:15'


('TR9199',
 '11:32',
 'monday',
 'broxbourne',
 'cambridge',
 '60 minutes',
 '10:32',
 '17.90 pounds')

In [4]:
model = GPT2LMHeadModel.from_pretrained("models/gpt2/multiwoz")

OSError: We couldn't connect to 'https://huggingface.co/' to load this model and it looks like models/gpt2/multiwoz is not the path to a directory conaining a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
sizencode = 256
predicted = {}
for batch in tqdm(datasets["test"]):
    did = batch["id"].lower().rstrip(".json")
    utterances = batch["text"].split("<sos_r>")
    predicted[did] = []
    responses = []
    for i in range(len(utterances)-1):
        example = "<sos_r>".join(utterances[:i+1])[-sizencode:]
        responses.append(example)
    encode = tokenizer(responses, return_tensors="pt", truncation=True,
                        padding=True, max_length=sizencode)
    gen = model2.generate(
        **encode,
        max_new_tokens=80,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.encode("<eos_r>")[0]
    )
    for response in gen:
        response = tokenizer.decode(response)
        predicted[did].append({
            "response": response.split("<sos_r>")[-1].split("<eos_r>")[0].strip(),
            # TODO
            # "state": response.split("<sos_b>")[-1].split("<eos_b>")[0].strip(),
            })

In [ ]:
from mwzeval.metrics import Evaluator
e = Evaluator(bleu=True, success=False, richness=True)
results = e.evaluate(predicted)
print(results)

In [41]:
text = "<sos_u> I need a train going to Cambridge that will depart after 10:15 from broxbourne. <eos_u>"
for i in range(80):
    example = tokenizer(text, return_tensors="pt")
    logits = model(**example).logits[0]
    text += tokenizer.decode(torch.argmax(logits[-1]))
print(text)

<sos_u> I need a train going to Cambridge that will depart after 10:15 from broxbourne. <eos_u><sos_b>[find_train]train-departure broxbourne train-destination cambridge train-leaveat 10:15<eos_b><sos_a>[train-inform] choice many [train-request] day?<eos_a><sos_r>There are <value_choice> trains that meet your criteria. What day would you like to travel?<eos_r><sos_u>I would like to leave on Tuesday.<eos_u><sos_b>[find_train]train-day t
